In [ ]:
import os, sys
os.chdir("/home/EPIC/src")
import CalculateCoElutionScores as CS

from ipywidgets import widgets, interact, interactive
from IPython.display import display
val = 0
def f(**kwargs):
    return None

features_i = interactive(f, MI = False, Bayes=False, Euclidean=False, WCC=False, Jaccard=False, PCCN=False, PCC=False, Apex=False)
directoryName_i = interactive(f, directory="input directory")
num_cores_i  = interactive(f, num_cores="1")
clf_i = interactive(f, classifier = ("random forest", "SVM"))
target_species_i = interactive(f, target_species= "taxid i.e. 6239 (Worm)")
display(features_i)
display(directoryName_i)
display(num_cores_i)
display(clf_i)
display(target_species_i)

In [ ]:
featuer_selection = [features_i.kwargs[feature_name] for feature_name in ['PCC', 'Jaccard', 'Apex', 'MI', 'Euclidean',  'WCC', 'Bayes', 'PCCN'] ]
num_cores = int(num_cores_i.kwargs['num_cores'])
root_dir = "/home/"
name = directoryName_i.kwargs['directory']
input_dir = root_dir + 'input' + os.sep + name 
output_dir = root_dir + 'output' + os.sep + name 
if os.path.exists(output_dir) == False: 
    os.mkdir(output_dir)
else:
    print "Output dir already exists"
    sys.exit()
output_dir += os.sep + "out"
target_taxid = target_species_i.kwargs['target_species']
use_rf = clf_i.kwargs['classifier'] != 'SVM'
if use_rf:
    output_dir += ".rf"
else:
    output_dir += ".svm"
print num_cores
print(featuer_selection)
print use_rf
print target_taxid
print input_dir
print output_dir

In [ ]:
all_scores = [CS.Pearson(), CS.Jaccard(), CS.Apex(), CS.MutualInformation(2), CS.Euclidiean(), CS.Wcc(), CS.Bayes(3), CS.Poisson(5)]
this_scores = []
for i, selection in enumerate(featuer_selection):
	if selection: this_scores.append(all_scores[i])
print this_scores
foundprots, elution_datas = CS.load_data(input_dir, this_scores)

In [ ]:
print len(foundprots)

In [ ]:
evals = CS.create_goldstandard(target_taxid, foundprots)
training_p, training_n, all_p, all_n, go_complexes, corum_complexes = evals

In [ ]:
scoreCalc = CS.CalculateCoElutionScores()
scoreCalc.addLabels(all_p, all_n)
scoreCalc.calculate_coelutionDatas(elution_datas, this_scores, output_dir, num_cores)

In [ ]:
scoreCalc.addLabels(training_p, training_n)
CS.bench_scores(scoreCalc, output_dir, num_cores, useForest=use_rf)

In [ ]:
from IPython.display import Image
Image(filename=output_dir + ".cutoff.png")

In [ ]:
Image(filename=output_dir + ".pr.png")

In [ ]:
Image(filename=output_dir + ".roc.png")

In [ ]:
scoreCalc.addLabels(all_p, all_n)
CS.predictInteractions(scoreCalc, output_dir , use_rf, num_cores)

In [ ]:
import GoldStandard as GS
predF = "%s.pred.txt" % (output_dir)
clustering_CMD = "java -jar cluster_one-1.0.jar %s > %s.clust.txt" % (predF, output_dir)
print(clustering_CMD)
os.system(clustering_CMD)

In [ ]:
CS.clustering_evaluation(evals, scoreCalc, output_dir, ",".join([score.name for score in this_scores]), num_cores, use_rf)